In [21]:
from file_process import * #==> issues on the import of splitDocs, don't read efficiently the content
## Fonction de traintement du texte
from text_process import *
from weiting_function import *
from runs_function import *
import re

In [14]:
list_file = list_file_data("../Practice_05_data/XML-Coll-withSem_/")
dataset = "../Practice_05_data/XML-Coll-withSem_/"
# Cell to initialise global variable
team_name = "DjibrilMohamedOmaimaDouae"
assets = "assets/"
k = 1.2
b = 0.75
use_stem = False
use_stopword = False
run_id=0
list_termes = {}

In [61]:
list_file[127]
fileName = '612.xml'
filePath = dataset+fileName

In [59]:
content = preprocesFile(dataset+fileName)


../Practice_05_data/XML-Coll-withSem_/612.xml


In [42]:
article = re.search('<article xmlns:xlink="http://www.w3.org/1999/xlink">(.*?)^[+&nbsp]</article>', str(content))
article

In [76]:
#from lxml import etree as ET
from lxml import html as ET

import xml.etree.ElementTree as ET2
article = ET.parse(dataset+fileName)
root = article.getroot()
for child in root:
    print(child.tag,child.attrib)

body {}


In [79]:
 with open(filePath, "r",encoding='utf-8') as file:
      # Read each line in the file, readlines() returns a list of lines
      content = file.readlines()
      # Combine the lines in the list into a string
      content = " ".join(content)
      soup = BeautifulSoup(content, "xml")
      text =  soup.get_text()
      article = soup.find_all('article')

In [81]:
header = soup.find_allrticle

In [3]:
def xml_text_minings_full(list_terms = dict(), use_stem=bool(), use_stopword=bool()):
    start = time.time()
    new_list_terms = {}
    posting_list = {}
    dl = list()
    i = 0
    for docno, content in list_terms.items():
        i+=1
        text_clean = clean(content, use_stem, use_stopword)
        new_list_terms[docno] = text_clean.split()
        lt = text_clean.split()
        current_dico = countWord(lt)
        #sec_1 = "[word : fre]"
        posting_list = countWordIntoDocs(current_dico,docno, posting_list)
    
    return posting_list, new_list_terms,(time.time() - start)

def xml_text_minings(list_terms = dict(), use_stem=bool(), use_stopword=bool()):
    start = time.time()
    new_list_terms = {}
    posting_list = {}
    posting_list_el = {}
    lt_ = dict()
    lt_final = dict()
    dl = list()
    i = 0

    for docno, content in list_terms.items():
        i+=1
        j = 0
        #print(i,"==>",id, "len sec==>", len(content))
        # if content[0] = 1 else normal indexion
        posting_list_el.clear()
        lt_.clear()
        for text in content:
            j+=1
            soup = BeautifulSoup(str(text), "xml")
            text_content = soup.get_text()
            #print(text_content)
            text_clean = clean(text_content, use_stem, use_stopword)
            #new_list_terms[id] = text_clean.split()
            lt = text_clean.split()
            lt_[j] = lt
            current_dico = countWord(lt)
            #sec_1 = "[word : fre]"
            posting_list_el = countWordIntoDocs(current_dico,"sec_{}".format(j), posting_list_el)
            #posting_list.setdefault(docno,[]).append(countWordIntoDocs(current_dico,"sec_{}".format(j), posting_list_el))
            #posting_list_el = countWordIntoDocs(current_dico,"sec_{}".format(j), posting_list_el) 
            #print("Sec[{}]==>{}".format(j,text_content))
            #print("**********-/*-*/-*-************************************************-/-*/-/-*/-*/-",j)
            
        lt_final[docno] = lt_.copy()
        posting_list[docno] = posting_list_el.copy();
     
        #break
        if(i==80): break
    return posting_list,lt_final,(time.time() - start)

def splitDocs3(fileDoc, list_terme):
    content = []
    sec_dict = {} # keys = {chiffre incrementale} # values = {contenues des paragraphes}
      # Read the XML file
    with open(fileDoc, "r",encoding='utf-8') as file:
      # Read each line in the file, readlines() returns a list of lines
      content = file.readlines()
      # Combine the lines in the list into a string
      content = " ".join(content)
      soup = BeautifulSoup(content, "xml")
      text =  soup.get_text()
      id = str(soup.find_all('id')[0]).strip('</id>')
      sec = soup.find_all('sec')
      # list_terme[id] = ['P'|'S'] # 0==> il n'ya pas de section et le 1 il y a des sections
   
      list_terme[id]=sec  # Pour chaque article on recuper les sections
    return list_terme

# For all weigting function
def all_score_wf(query_list, ltn,ltc,bm25):
    start = time.time()
    score_ltn= score(query,ltn)
    print("Ltn time : ",time.time() -start)
    start = time.time()
    score_ltc = score(query,ltc)
    print("Ltc time : ",time.time() -start)
    start = time.time()
    score_bm25 = score(query,bm25)
    print("Bm25 time : ",time.time() -start)
    
    return score_ltn,score_ltc,score_bm25

In [4]:
# For full content
for file_name in list_file:
    list_termes = splitDocs2(dataset+file_name,list_termes)
pl_f,lt_f,ti = xml_text_minings_full(list_termes,use_stem, use_stopword)

print(ti/60)
stat_ = get_statistics(pl_f, lt_f)

0.02127832571665446


In [4]:
# For content with sec
for file_name in list_file:
    list_termes = splitDocs3(dataset+file_name,list_termes)
pl,lt,ti = xml_text_minings(list_termes,use_stem, use_stopword)
new_pl = {}

print(ti/60)


0.08189438581466675


In [5]:

new_pl = {}
for dic, values in pl.items():
        for word,v in values.items():
            new_pl.setdefault(word,[]).append({dic:v})


In [6]:

new_pl['food']

[{'10136': [('sec_3', 2)]},
 {'1064': [('sec_2', 1), ('sec_3', 1), ('sec_8', 1), ('sec_12', 1)]},
 {'10646': [('sec_1', 12),
   ('sec_2', 2),
   ('sec_3', 28),
   ('sec_4', 33),
   ('sec_5', 17),
   ('sec_6', 18),
   ('sec_7', 4),
   ('sec_8', 2),
   ('sec_9', 5),
   ('sec_10', 3),
   ('sec_11', 5),
   ('sec_12', 25)]},
 {'10843': [('sec_4', 1), ('sec_5', 1), ('sec_6', 5)]},
 {'10975': [('sec_1', 1), ('sec_7', 1)]},
 {'11243': [('sec_8', 1)]},
 {'11286': [('sec_1', 1), ('sec_3', 3), ('sec_6', 3), ('sec_7', 2)]},
 {'11298': [('sec_1', 1)]},
 {'1134': [('sec_1', 1)]},
 {'1164': [('sec_8', 2)]},
 {'12424': [('sec_8', 1)]},
 {'12436': [('sec_3', 1), ('sec_8', 10)]},
 {'12486': [('sec_1', 4),
   ('sec_3', 2),
   ('sec_4', 1),
   ('sec_5', 2),
   ('sec_6', 1),
   ('sec_7', 2)]},
 {'14647': [('sec_3', 2), ('sec_5', 2)]},
 {'1841': [('sec_1', 2), ('sec_2', 2)]}]

In [7]:
# Part of statistique

def doc_len_xml(list_terms):
    dl = {}
    dl_sec = {}
    n_sec = 0
    for doc_n, sec in list_terms.items():
        dl_sec.clear()
        n_sec+=len(sec)
        for sec_n, len_sec in sec.items():
            dl_sec[sec_n] = len(len_sec)
        dl[doc_n] = [dl_sec.copy()]
    # dl = [(doc_n, len(len_doc)) for doc_n,len_doc in list_terms.items()]
    return dl,n_sec
#{docn : {sec_1:5}}

def collection_term_freq_xml(posting_list):
    c_size = {}
    dl = {}
    for term, article in posting_list.items():  # get the term
        somme = 0
        j = 0
        for secs in article:
            
            for sec,val in secs.items():
                
                for v in val:
                    j+=1
                    somme += v[1]
        dl[term] = [(somme, len(article),j)]
                                                   # somme = somme de frequence dans toutes les sections
    return dl                                              
                                                            # j = nombre total d'element
def get_xml_stat(posting_list, list_terms):
    stat = {}
    # document length
    stat['df'],stat['n_sec'] = doc_len_xml(list_terms)
    # Number of a doc
    stat['n_article'] = len(stat['df']) # n article
    stat['n_element'] = stat['n_sec'] + stat['n_article']
    # term length
    stat['tl'] = term_len(posting_list)
    # vocabulary size
    stat['voc_size'] = vocabulary_size(posting_list)
    # collection frequency of terms
    stat['colec_freq'] = collection_term_freq_xml(posting_list)
    return stat

In [8]:
stat = get_xml_stat(new_pl,lt)

In [9]:
"""
   i=0
      # '10013' = ['P'|'S',[list des paragraphes],[listes de sections(listes des paragraphes)]]
      if(len(sec)==0):
        p = ['P']
        p.append(soup.find_all('p'))
        list_terme[id] = p

    else : 
        sec = ['S']

        for paragraphe in sec:
            i+=1
            soup = BeautifulSoup(paragraphe, "xml")
            p = soup.find_all('p')
            sec_dict[i] = p
"""

'\n   i=0\n      # \'10013\' = [\'P\'|\'S\',[list des paragraphes],[listes de sections(listes des paragraphes)]]\n      if(len(sec)==0):\n        p = [\'P\']\n        p.append(soup.find_all(\'p\'))\n        list_terme[id] = p\n\n    else : \n        sec = [\'S\']\n\n        for paragraphe in sec:\n            i+=1\n            soup = BeautifulSoup(paragraphe, "xml")\n            p = soup.find_all(\'p\')\n            sec_dict[i] = p\n'

Article > Sections > Paragraphes
        >Paragraphes


In [15]:
# Part of weigting function
def smart_ltn_xml(posting_list,stat):
    i = 0
    n = stat['n_element']
    df_t = stat['colec_freq']
    df = 0
    #(1+LOG(tf))*LOG(n/df)
    #tf is the frequence of term on a document
    #df number of document where the term appear
    ltn_value_sec = {} # key sec, value (terme, ltn)
    ltn_value = {} # key doc number, value (terme, ltn)
    for term, articles in posting_list.items():
        i+=1
        dl = df_t[term]
        df = dl[0][2] # nombre de doc dans lequel le terme apparait sur toute la collection
        # print(value) [{'10013': [('sec_1', 1), ('sec_12', 1)]},...]
        
        for sec in articles:
            # print(sec.values()) dict_values([[('sec_1', 1), ('sec_12', 1)]])
            
            for doc_no,val in sec.items():
                ltn_value_sec.clear()
                for v in val:
                    tf = v[1]
                    sec =v[0]
                    ltn = (1+math.log10(tf))*(math.log10(n/df))
                    ltn_value_sec.setdefault(sec,[]).append((term,round(ltn,4))) # ltn en fonction de la section
           
                ltn_value.setdefault(doc_no,[]).append(ltn_value_sec.copy()) # ltn en fonction des articles
    return ltn_value

def ltn_ref(ltn):
    new_ltn = {}
    ltn_refractor = {}
    for doc, article in ltn.items():
        new_ltn.clear()
        for l in article: 
            for se, k in l.items():
                for i in k:
                    new_ltn.setdefault(se,[]).append(i)
        ltn_refractor[doc] = new_ltn.copy()
    return ltn_refractor
query = "olive from oil health benefit"
def xml_w_score(wf=dict()):
    score_wt = {}
    for doc_no, article in wf.items():
        scores = rsv_score(query,article)
        if(len(scores)>0):
            for sec in scores:
                score_wt.setdefault(doc_no,[]).append(sec)
    return score_wt

def ltc_xml(ltn):
    score_ltn = {}
    for doc_no, sec_ltn_value in ltn.items():
        for value in sec_ltn_value:
            res = smart_ltc(sec_ltn_value)
            score_ltn[doc_no] = res
    return score_ltn

In [16]:
#stat['n_element']
ltn = ltn_ref(smart_ltn_xml(new_pl,stat))
score_ltn = xml_w_score(ltn)
"""
ltn
{'10013': [{'sec_1': [('overview', 1.2433)],
   'sec_12': [('overview', 1.2433)]}],
 '10136': [{'sec_1': [('overview', 1.2433)]}],...]
"""
#score_ltn

"\nltn\n{'10013': [{'sec_1': [('overview', 1.2433)],\n   'sec_12': [('overview', 1.2433)]}],\n '10136': [{'sec_1': [('overview', 1.2433)]}],...]\n"

In [17]:
ltn_,ltc_,bm25_ =weinting_function(pl_f, stat_,k,b) 

In [18]:
ltc =ltc_xml(ltn)
score_ltc = xml_w_score(ltc)
#score_ltc

In [ ]:
score_ltn

In [29]:
def build_run_file_xml(run_id, wf_score, index, use_stem, use_stopword, k, b):
    start = time.time()
    if index != 2:
        k = 0.0
        b = 0.0
    run_file_name, run_id = create_run_file(run_id, index, use_stem, use_stopword, k, b)
    for article, scores in wf_score.items():
        for score in scores:
            print(len(score))
            for i in range(len(score)):
                run_score = str(score[i]).replace(',', '').replace("'", '').replace('(', '').replace(')', '')
                print(run_score)
                break
                run_file_name.write(run_score + "\n")
        run_file_name.close()
        break
    print("Execution time for the run {} is {}".format(run_id, time.time() - start))
    return run_id


In [128]:

def scores(query_list,rsv_wf,docno,j):
    team_name = "DjibrilMohamedOmaimaDouae"
    score = []
    k=j
    for query in query_list:
        
        query = query.split(' ', 1)
        result,k = result_querys(query[0],docno, rsv_score(query[1], rsv_wf), team_name,k)
        score.append(result)
    return score,k

def result_querys(num_query, docno, rsv_result,team_name, j):
    start = time.time()
    score = reverse_score(rsv_result)
    rsv_r = []
    k = 0
    x = score
   
    if len(x)>1500:
        x = x[:1500]
        
    for i in range(len(x)):
        k = i+j
        rsv_r.append(('-',num_query,'Q0', '{}'.format(docno),k,
                      round(x[i][1],5), team_name,'/article[1]/sec[{}]'.format(x[i][0].replace("sec_",""))))
    return rsv_r,k
# 2009011 Q0 1164 1 0.7262 DjibrilMohamedOmaimaDouae /article[1]/sec

In [129]:
i = 0
ltn_s = []
j = 1
for docno, a in ltn.items():
    i+=1
    res,j = scores(query,a,docno,j)
    if(len(res)>0):
        ltn_s.append(res)
    print(ltn_s)
    if i==10:
        break
run_id = build_run_files(run_id, ltn_s, 0, use_stem, use_stopword, k, b)

#score_ltn_
#build_run_file_xml(run_id, score_ltn, 0, use_stem, use_stopword, k, b)
#ltn_test
#article[1]/sec[1]

[[[('-', '2009011', 'Q0', '10013', 1, 1.2541, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[2]'), ('-', '2009011', 'Q0', '10013', 2, 1.6316, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[11]'), ('-', '2009011', 'Q0', '10013', 3, 1.6316, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[3]'), ('-', '2009011', 'Q0', '10013', 4, 1.6316, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[7]'), ('-', '2009011', 'Q0', '10013', 5, 1.68, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[6]'), ('-', '2009011', 'Q0', '10013', 6, 2.0091, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[12]'), ('-', '2009011', 'Q0', '10013', 7, 2.9341, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[8]'), ('-', '2009011', 'Q0', '10013', 8, 3.4399, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[4]')], [], [('-', '2009067', 'Q0', '10013', 0, 0.185, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[10]'), ('-', '2009067', 'Q0', '10013', 1, 0.185, 'DjibrilMohamedOmaimaDouae', '/article[1]/sec[7]'), ('-', '2009067', 'Q0', '10013', 2, 0.185, 

In [35]:
# get the query file
directory = "../Practice_04_data/"
#list_data_q = practice2.list_file_data(directory)
filename_q = str(directory+"topics_M2DSC_7Q.txt")
query_list =preprocesFile(filename_q)
query = query_list.split("\n")

../Practice_04_data/topics_M2DSC_7Q.txt


In [21]:
ltn_,ltc_,bm25_ =weinting_function(pl_f, stat_,k,b) 
score_ltn_,score_ltc_,score_bm25_ = all_score_wf(query, ltn_,ltc_,bm25_)

Ltn time :  0.6507244110107422
Ltc time :  0.39051151275634766
Bm25 time :  0.39559245109558105


In [97]:
# index is [0,1,2]==['ltn','ltc','bm25']
def build_run_files(run_id, wf_score, index, use_stem, use_stopword, k, b):
    start = time.time()
    if index != 2:
        k = 0.0
        b = 0.0
    run_file_name, run_id = create_run_file(run_id, index, use_stem, use_stopword, k, b)

    for score in wf_score:
        for i in range(len(score)):
            if(len(score[i])>0):
                run_score = ''(score[i]).replace(',', '').\
                replace("'", '').replace('(', '').\
                replace(')', '').replace(' ]','').\
                replace("[- ","").replace(' - ',"\n")
            if score[i]==score[-1]:
                run_score = run_score.replace("]]","")
            print(run_score)
            break
            run_file_name.write(run_score + "\n")
    run_file_name.close()

    print("Execution time for the run {} is {}".format(run_id, time.time() - start))
    return run_id

In [45]:

sorted([["10","sec1",0.3],["11","sec1",0.4],["12","sec1",0.1],["10","sec2",0.1]])

TypeError: unhashable type: 'list'